# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!


## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it.

You will find more info about virtual environments in the course content and on the web.

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../01-TheField/02-Python/02-PythonAdvanced/05-Scraping/03-apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [11]:
# import the requests library (1 line)
import requests
# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = "https://country-leaders.onrender.com"
# assign the /status endpoint to another variable called status_url (1 line)
status_url = f"{root_url}/status"
# query the /status endpoint using the get() method and store it in the req variable (1 line)
r = requests.get(status_url)
# check the status_code using a condition and print appropriate messages (4 lines)
if r.status_code == 200:
    print(r.text)
else:
    print(r.status_code)

"Alive"


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [12]:
# Set the countries_url variable (1 line)
countries_url = f"{root_url}/countries"
# query the /countries endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(countries_url)
# Get the JSON content and store it in the countries variable (1 line)
countries = req.json()
# display the request's status code and the countries variable (1 line)
print(req.status_code, countries)

403 {'message': 'The cookie is missing'}


### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [13]:
# Set the cookie_url variable (1 line)
cookie_url = f'{root_url}/cookie'
# Query the endpoint, set the cookies variable and display it (2 lines)
req_cookie = requests.get(cookie_url)
cookies = req_cookie.cookies
print(req_cookie.cookies)

<RequestsCookieJar[<Cookie user_cookie=ea31729d-774f-4d4c-9ba8-73115fd920c3 for country-leaders.onrender.com/>]>


Try to query the countries endpoint using the cookie, save the output and print it.

In [14]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries = requests.get(countries_url).json()
# display the countries variable (1 line)
request = requests.get(countries_url, cookies = cookies)
print(request.json())



['ma', 'us', 'ru', 'be', 'fr']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [15]:
# Set the leaders_url variable (1 line)
leaders_url = f"{root_url}/leaders"
# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders = requests.get(leaders_url).json()
# display the leaders variable (1 line)
req_leaders = requests.get(leaders_url, cookies = cookies)
print(req_leaders.json())

{'message': 'Please specify a country'}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [16]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
response_leaders_US = requests.get(leaders_url, cookies = cookies, params = "country=us")
# assign the output to the leaders variable (1 line)
leaders_US = response_leaders_US.json()
# display the leaders variable (1 line)
print(leaders_US)

[{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'place_of_bi

### A sneak peak at the data (finally)


Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [17]:
# 4 lines
#initialize an empty dictionary to store the results: 
leaders_per_country = {}
countries = request.json()
print(countries)
#Loop through the list of country codes and query the /leaders endpoint for each one:
for country_code in countries:
    params = {"country": country_code}
    response_leaders = requests.get(leaders_url, params=params, cookies=cookies)
    leaders_per_country[country_code] = response_leaders.json()
print(leaders_per_country)

['ma', 'us', 'ru', 'be', 'fr']
{'ma': [{'id': 'Q57553', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1963-08-21', 'death_date': None, 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86', 'start_mandate': '1999-07-23', 'end_mandate': None}, {'id': 'Q69103', 'first_name': 'Hassan', 'last_name': 'None', 'birth_date': '1929-07-09', 'death_date': '1999-07-20', 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF', 'start_mandate': '1961-02-26', 'end_mandate': '1999-07-23'}, {'id': 'Q193874', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1909-08-10', 'death_date': '1961-02-26', 'place_of_birth': 'Fez', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D

In [8]:
# or 1 line


It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [19]:
# < 15 lines
import requests
def get_leaders():
    root_url = "https://country-leaders.onrender.com"
    countries_url = f"{root_url}/countries"
    leaders_url = f"{root_url}/leaders"
    cookie_url = f'{root_url}/cookie'
    req_cookie = requests.api.get(cookie_url)
    cookies = req_cookie.cookies
    response_countries = requests.get(countries_url, cookies=cookies)
    countries = response_countries.json()
    leaders_per_country = {}
    for country_code in countries:
        params = {"country": country_code}
        response_leaders = requests.get(leaders_url, params=params, cookies=cookies)
        leaders_per_country[country_code] = response_leaders.json()
    return leaders_per_country


Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [20]:
# 2 lines
leaders_data = get_leaders() 
print(leaders_data)

{'ma': [{'id': 'Q57553', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1963-08-21', 'death_date': None, 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86', 'start_mandate': '1999-07-23', 'end_mandate': None}, {'id': 'Q69103', 'first_name': 'Hassan', 'last_name': 'None', 'birth_date': '1929-07-09', 'death_date': '1999-07-20', 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF', 'start_mandate': '1961-02-26', 'end_mandate': '1999-07-23'}, {'id': 'Q193874', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1909-08-10', 'death_date': '1961-02-26', 'place_of_birth': 'Fez', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3_%D8%A8%D9%86_%D9%8A%D9%88%

## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [21]:
# 3 lines
country_code = list(leaders_data.keys())[4]
leader_info = leaders_data[country_code][1]

wikipedia_url = leader_info["wikipedia_url"]
print(f"Wikipedia URL: {wikipedia_url}")

Wikipedia URL: https://fr.wikipedia.org/wiki/Nicolas_Sarkozy


Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [22]:
# 3 lines
from bs4 import BeautifulSoup
response_wiki = requests.get(wikipedia_url)
soup = BeautifulSoup(response_wiki.content, 'html.parser')
pretty_content = soup.prettify()
print(pretty_content)

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-enabled vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="fr">
 <head>
  <meta charset="utf-8"/>
  <title>
   Nicolas Sarkozy — Wikipédia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disab

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [23]:
# 2 lines
paragraphs = soup.find_all('p')
for para in paragraphs:
    text = para.get_text().strip()
    print(text)
    


« Sarkozy » redirige ici. Pour les autres significations, voir Sarkozy (homonymie), Nagy et Bocșa.

Nicolas Sarközy de Nagy-Bocsa[d], dit Nicolas Sarkozy (/ni.kɔ.la saʁ.kɔ.zi/[e] ; Écouter), né le 28 janvier 1955 à Paris (France), est un homme d'État français. Il est président de la République française du 16 mai 2007 au 15 mai 2012.
Il occupe d'abord les fonctions de maire de Neuilly-sur-Seine, député, ministre du Budget et porte-parole du gouvernement ou encore de président par intérim du Rassemblement pour la République (RPR). À partir de 2002, il est ministre de l'Intérieur (à deux reprises), ministre de l'Économie et des Finances et président du conseil général des Hauts-de-Seine. Il est alors l'un des dirigeants les plus en vue de l'Union pour un mouvement populaire (UMP), qu'il préside de 2004 à 2007.
Élu président de la République française en 2007 avec 53,1 % des suffrages face à Ségolène Royal, il inaugure une rupture de style et de communication par rapport à ses prédécesse

If you try different urls, you might find that the paragraph you want may be at a different index each time.


That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...
Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [24]:
# <10 lines
for para in paragraphs:
    text = para.get_text().strip()
    if para.find('b') or para.find('strong'):
        first_paragraph = text
        print(first_paragraph)
    



Nicolas Sarközy de Nagy-Bocsa[d], dit Nicolas Sarkozy (/ni.kɔ.la saʁ.kɔ.zi/[e] ; Écouter), né le 28 janvier 1955 à Paris (France), est un homme d'État français. Il est président de la République française du 16 mai 2007 au 15 mai 2012.


At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [85]:
# 10 lines
from bs4 import BeautifulSoup

ountry_code = list(leaders_data.keys())[4]
leader_info = leaders_data[country_code][1]
wikipedia_url = leader_info["wikipedia_url"]
print(f"Wikipedia URL: {wikipedia_url}") 

def get_first_paragraph(wikipedia_url): 
    response_wiki = requests.get(wikipedia_url)
    soup = BeautifulSoup(response_wiki.content, 'html.parser')
    paragraphs = soup.find_all('p')

    for para in paragraphs:
        text = para.get_text().strip()
        if para.find('b') or para.find('strong'):
            return text
        
first_paragraph = get_first_paragraph(wikipedia_url)
print(first_paragraph)    # keep this for the rest of the notebook

#   [insert your code]
#   return first_paragraph

Wikipedia URL: https://en.wikipedia.org/wiki/Barack_Obama
Barack Hussein Obama II[a] (born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. As a member of the Democratic Party, he was the first African-American  president in United States history. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008, as an Illinois state senator from 1997 to 2004, and as a community service organizer, civil rights lawyer, and university lecturer.


In [16]:
# Test: 3 lines


### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [92]:
# 3 lines
import re

def sanitize_text(first_paragraph):
    return re.sub(r'\[.*?\]|<.*?>|/.*?/', '', text)
    
    # Remove references (e.g., [1], [2], [3])
   
    # Remove any HTML tags
    
    # Remove phonetic pronunciation (e.g., /fəˈnɛtɪk/)
    
 



Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [18]:
# 10 lines


Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [93]:
# < 20 lines
def get_first_paragraph(wikipedia_url): 
    response_wiki = requests.get(wikipedia_url)
    soup = BeautifulSoup(response_wiki.content, 'html.parser')
    paragraphs = soup.find_all('p')
    
    for para in paragraphs:
        text = para.get_text().strip()
        if para.find('b') or para.find('strong'):
            return re.sub(r'\[.*?\]|<.*?>|/.*?/', '', text).strip()

# Fetch leaders data
leaders_data = get_leaders()

# Example usage with one of the leaders' Wikipedia URLs
country_code = list(leaders_data.keys())[4]
leader_info = leaders_data[country_code][1]
wikipedia_url = leader_info["wikipedia_url"]
print(f"Wikipedia URL: {wikipedia_url}") 

# Get and print the sanitized first paragraph from the Wikipedia page
first_paragraph = get_first_paragraph(wikipedia_url)
print(first_paragraph)

Wikipedia URL: https://en.wikipedia.org/wiki/Barack_Obama
Barack Hussein Obama II (born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. As a member of the Democratic Party, he was the first African-American  president in United States history. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008, as an Illinois state senator from 1997 to 2004, and as a community service organizer, civil rights lawyer, and university lecturer.


## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [96]:
# < 20 lines
import requests
from bs4 import BeautifulSoup
import re

def get_leaders():
    root_url = "https://country-leaders.onrender.com"
    countries_url = f"{root_url}/countries"
    leaders_url = f"{root_url}/leaders"
    cookie_url = f'{root_url}/cookie'
    req_cookie = requests.api.get(cookie_url)
    cookies = req_cookie.cookies
    response_countries = requests.get(countries_url, cookies=cookies)
    countries = response_countries.json()
    leaders_per_country = {}
    for country_code in countries:
        params = {"country": country_code}
        response_leaders = requests.get(leaders_url, params=params, cookies=cookies)
        leaders_per_country[country_code] = response_leaders.json()
    return leaders_per_country


def get_first_paragraph(wikipedia_url): 
    response_wiki = requests.get(wikipedia_url)
    soup = BeautifulSoup(response_wiki.content, 'html.parser')
    paragraphs = soup.find_all('p')
    
    for para in paragraphs:
        text = para.get_text().strip()
        if para.find('b') or para.find('strong'):
            return re.sub(r'\[.*?\]|<.*?>|/.*?/', '', text)

# Fetch leaders data
leaders_data = get_leaders()

# Example usage with one of the leaders' Wikipedia URLs
country_code = list(leaders_data.keys())[4]
leader_info = leaders_data[country_code][1]
wikipedia_url = leader_info["wikipedia_url"]
print(f"Wikipedia URL: {wikipedia_url}") 
    

Wikipedia URL: https://en.wikipedia.org/wiki/Barack_Obama


In [97]:
# Check the output of your function (2 lines)
first_paragraph = get_first_paragraph(wikipedia_url)
print(first_paragraph)

Barack Hussein Obama II (born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. As a member of the Democratic Party, he was the first African-American  president in United States history. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008, as an Illinois state senator from 1997 to 2004, and as a community service organizer, civil rights lawyer, and university lecturer.


Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [22]:
# < 25 lines



Check the output of your function again.

In [23]:
# Check the output of your function (1 line)


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [98]:
# < 20 lines
def get_first_paragraph(wikipedia_url, session): 
    response_wiki = session.get(wikipedia_url)
    soup = BeautifulSoup(response_wiki.content, 'html.parser')
    paragraphs = soup.find_all('p')
    
    for para in paragraphs:
        text = para.get_text().strip()
        if para.find('b') or para.find('strong'):
            return re.sub(r'\[.*?\]|<.*?>|/.*?/', '', text).strip()

# Fetch leaders data
leaders_data = get_leaders()

# Create a session
with requests.Session() as session:
    # Iterate over each country
    for country_code, leaders_info in leaders_data.items():
        print(f"Country Code: {country_code}")
        # Iterate over each leader for the current country
        for leader_info in leaders_info:
            wikipedia_url = leader_info["wikipedia_url"]
            print(f"Wikipedia URL: {wikipedia_url}")
            # Get and print the sanitized first paragraph from the Wikipedia page
            first_paragraph = get_first_paragraph(wikipedia_url, session)
            print(first_paragraph)
            print()# Separate each leader's output for clarity

Country Code: ma
Wikipedia URL: https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86
مُحمد السادس بن الحسن الثاني العلوي (مواليد 21 أغسطس 1963) هو ملك المملكة المغربية منذ عام 1999 والملك الثالث والعشرون للمغرب من سلالة العلويين الفيلاليين، تولى الحكم خلفًا لوالده الملك الحسن الثاني بعد وفاته، وبويع ملكًا يوم الجمعة 9 ربيع الثاني سنة 1420 هـ الموافق 23 يوليو 1999 بالقصر الملكي بالرباط.

Wikipedia URL: https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF
الحَسَن الثاني بِنْ مُحمد بِنْ يوسف العَلوي (9 يوليو 1929 – 23 يوليو 1999)، ثاني ملوك المملكة المغربية بعد الإستقلال، والملك الثاني والعشرين للمغرب من سلالة العلويين الفيلاليين، تولى حكم المملكة المغربية خلفًا لوالده الملك محمد الخامس في 26 فبراير 1961 وحتى وفاته في 20 يوليو 1999. ينتمي الملك الحسن الثاني إلى السلالة العلوية التي تعود في نسبها إلى الحسن بن علي بن أبي طالب، 

Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [25]:
# <25 lines


Test your new functions.



## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [25]:
# 3 lines
import json

with open('leader.json', 'w') as f:
    json.dump(leaders_per_country, f)

Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [26]:
# 3 lines
with open('leader.json') as f:
    leaders_per_country_copy = json.load(f)

print(leaders_per_country_copy == leaders_per_country)

True


Make a function `save(leaders_per_country)` to call this code easily.

In [27]:
# 3 lines
def save(leaders_per_country):
    with open('leader.json', 'w') as f:
        json.dump(leaders_per_country, f)

In [28]:
# Call the function (1 line)
save(leaders_per_country)

## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!